**Created by Sanskar Hasija**

**🤖H2O.ai AutoML Classification - Titanic**

**15 NOVEMBER 2021**


# <center> 🤖H2O.AI AUTOML CLASSIFICATION - TITANIC</center>
## <center>If you find this notebook useful, support with an upvote👍</center>

# Imports

In [ ]:
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

# Initializing H20 Cluster

In [ ]:
from h2o.automl import H2OAutoML
h2o.init()

# Data Loading and Preprocessing

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv('../input/titanic/test.csv')
subs = pd.read_csv('../input/titanic/gender_submission.csv')

drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)

str_list = [] 
num_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)
        
train = pd.get_dummies(train, columns=str_list)
test = pd.get_dummies(test, columns=str_list)

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

In [ ]:
train.describe()

# AutoML

In [ ]:
MAX_RUNTIME = 1800 #30 mins
RANDOM_SEED = 12 

y = "Survived"
train[y] = train[y].asfactor()
x = train.columns
x.remove(y)

In [ ]:
aml = H2OAutoML(seed = RANDOM_SEED , max_runtime_secs=MAX_RUNTIME)
aml.train(x = x, y = y, training_frame = train)

# Results Leaderboard

### Top 10 models

In [ ]:
lb = aml.leaderboard
lb.head(10)

### Submission

In [ ]:
preds = aml.leader.predict(test)
sub = preds[0].as_data_frame()  ## Converting to dataframe
subs['Survived'] = sub
subs.to_csv('h20.ai-automl.csv',index = False)